In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import joblib

In [34]:
# 1) Load the cleaned transactions
df = pd.read_csv("../data/transactions_clean.csv", parse_dates=["InvoiceDate"])
print("Loaded transactions:", df.shape)

# 2) Feature Engineering
# ──────────────────────
# Extract time features
df["hour"] = df["InvoiceDate"].dt.hour
df["day_of_week"] = df["InvoiceDate"].dt.dayofweek
df["month"] = df["InvoiceDate"].dt.month

Loaded transactions: (779495, 10)


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779495 entries, 0 to 779494
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      779495 non-null  int64         
 1   StockCode    779495 non-null  object        
 2   Description  779495 non-null  object        
 3   Quantity     779495 non-null  int64         
 4   InvoiceDate  779495 non-null  datetime64[ns]
 5   Price        779495 non-null  float64       
 6   Customer ID  779495 non-null  float64       
 7   Country      779495 non-null  object        
 8   TotalPrice   779495 non-null  float64       
 9   Month        779495 non-null  object        
 10  hour         779495 non-null  int32         
 11  day_of_week  779495 non-null  int32         
 12  month        779495 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(3), int64(2), object(4)
memory usage: 68.4+ MB


In [38]:
# Define features and target
target = "TotalPrice"
categorical_cols = ["StockCode", "Country"]
numerical_cols   = ["Quantity", "Price", "hour", "day_of_week", "month"]
features = categorical_cols + numerical_cols

# 3) Encode categoricals
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    joblib.dump(le, f"../models/enc_{col}.pkl")  # save encoders

In [39]:
# 4) Prepare data arrays
X = df[features].values
y = df[target].values.reshape(-1, 1)

In [40]:
# 5) Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Train size:", X_train.shape, "Test size:", X_test.shape)

Train size: (623596, 7) Test size: (155899, 7)


In [41]:
# 6) Scale numerical features
#    We scale only the numerical part of X
num_start = len(categorical_cols)
scaler = StandardScaler()
X_train[:, num_start:] = scaler.fit_transform(X_train[:, num_start:])
X_test[:, num_start:]  = scaler.transform(X_test[:, num_start:])
joblib.dump(scaler, "../models/scaler_num.pkl")

['../models/scaler_num.pkl']

In [43]:
# 7) Self‑Supervised Pretraining
# ────────────────────────────────
from pytorch_tabnet.callbacks import EarlyStopping

pretrainer = TabNetPretrainer(
    input_dim=X_train.shape[1],
    mask_type='entmax'  # sparsity in masks
)
# define callbacks
es = EarlyStopping(
    patience=20,
    early_stopping_metric="loss",  # Metric to monitor
    is_maximize=False              # Set to False for minimizing loss
)

# fit with callbacks and verbose logging
pretrainer.fit(
    X_train,
    max_epochs=2,
    batch_size=1024,
    virtual_batch_size=128,
    pretraining_ratio=0.8,
    num_workers=3,
    drop_last=False,
    callbacks=[es]
    
)

pretrainer.save_model("../models/tabnet_pretrainer")
print("✅ Pretraining complete.")

c:\Users\hp\Innovative-ml-project\NEW-ML-PROJECT\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
c:\Users\hp\Innovative-ml-project\NEW-ML-PROJECT\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 2.31245 |  0:00:41s
epoch 1  | loss: 1.39254 |  0:01:20s
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_loss = 1.39254
epoch 1  | loss: 1.39254 |  0:01:20s
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_loss = 1.39254


c:\Users\hp\Innovative-ml-project\NEW-ML-PROJECT\.venv\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at ../models/tabnet_pretrainer.zip
✅ Pretraining complete.


In [44]:
# 8) Fine‑Tuning as Regressor
# ────────────────────────────
regressor = TabNetRegressor(
    input_dim=X_train.shape[1],
    output_dim=1,
    optimizer_fn=torch.optim.Adam,
    optimizer_params={"lr":1e-3, "weight_decay":1e-4},
    scheduler_params={"step_size":50, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR
)

# load pretrained encoder weights
# Instead of regressor.load_weights_from_unsupervised(pretrainer)
# use the following:
regressor._set_network() # Initialize the network attribute
regressor.load_weights_from_unsupervised(pretrainer) # Load weights

c:\Users\hp\Innovative-ml-project\NEW-ML-PROJECT\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [45]:

# fit on train, evaluate on test
regressor.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_name=['test'],
    max_epochs=2,
    patience=30,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

epoch 0  | loss: 53419.30011| test_mse: 35898.09669|  0:00:30s
epoch 1  | loss: 52088.24793| test_mse: 21379.15374|  0:00:59s
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_test_mse = 21379.15374
epoch 1  | loss: 52088.24793| test_mse: 21379.15374|  0:00:59s
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_test_mse = 21379.15374


c:\Users\hp\Innovative-ml-project\NEW-ML-PROJECT\.venv\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [46]:
# Save the fine‑tuned model
regressor.save_model("../models/tabnet_regressor")
print("✅ Fine‑tuning complete. Model saved to models/tabnet_regressor.zip")

Successfully saved model at ../models/tabnet_regressor.zip
✅ Fine‑tuning complete. Model saved to models/tabnet_regressor.zip
